Import Library

In [1]:
import numpy as np
import cv2
import pandas as pd
import seaborn as sns
import os
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.optimizers import Adam, SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
import scipy

Create Data Generator

In [2]:
train_dir = 'data/train'
val_dir = 'data/val'
test_dir= 'data/test/'

num_train = 28709
num_val = 3589
num_test = 3589
batch_size = 128
num_epoch = 20

data_gen = ImageDataGenerator(rescale=1./255,rotation_range=10,
                              zoom_range=0.1,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              horizontal_flip=True)
train_generator= data_gen.flow_from_directory(train_dir,
                                    target_size=(48, 48),
                                    batch_size=batch_size,
                                    shuffle=True,
                                    color_mode="grayscale",
                                    class_mode='categorical')

validation_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_dir,
        target_size=(48, 48),
        batch_size=batch_size,
        shuffle=False,
        color_mode="grayscale",
        class_mode='categorical')

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_dir,
        target_size=(48, 48),
        batch_size=batch_size,
        shuffle=False,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [3]:
Model_Checkpoint_Callback = ModelCheckpoint('model.h5', 
                                            monitor='val_accuracy', 
                                            verbose=1, 
                                            save_best_only=True, 
                                            mode='max',
                                            save_freq = "epoch")

Model Building

In [4]:
model = Sequential()

model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(256,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(7, activation='softmax'))

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
'''history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=num_train//batch_size,
                              epochs=num_epoch,
                              validation_data=validation_generator,
                              validation_steps=num_val//batch_size,
                              callbacks=[Model_Checkpoint_Callback]
                              )'''
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 48, 48, 64)        256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 batch_normalization_1 (Bat  (None, 24, 24, 128)       5

Load Saved Model

In [7]:
from keras.models import load_model
model_test=load_model('model.h5')
model_test.evaluate(test_generator)

29/29 [==============================] - 7s 216ms/step - loss: 1.0970 - accuracy: 0.5982


[1.0969610214233398, 0.5982167720794678]

Create Confusion Matrix

In [8]:
from keras.models import load_model

model_test = load_model("model.h5")
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model_test.predict_generator(test_generator, num_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

C:\Users\Tommy\AppData\Local\Temp\ipykernel_27064\3668391031.py:5: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model_test.predict_generator(test_generator, num_test // batch_size+1)


Confusion Matrix
[[248  16  11  32  74  82   4]
 [ 19  19   0   1   5  12   0]
 [ 45   7 110  36  86 188  24]
 [ 17   0  10 760  70  28  10]
 [ 41   0   7  73 357 128   1]
 [ 67   1  14  48 119 402   2]
 [ 21   2  40  35  39  27 251]]


In [9]:
print('Classification Report')
target_names = list(train_generator.class_indices.keys())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

       angry       0.54      0.53      0.54       467
     disgust       0.42      0.34      0.38        56
        fear       0.57      0.22      0.32       496
       happy       0.77      0.85      0.81       895
     neutral       0.48      0.59      0.53       607
         sad       0.46      0.62      0.53       653
    surprise       0.86      0.60      0.71       415

    accuracy                           0.60      3589
   macro avg       0.59      0.54      0.54      3589
weighted avg       0.61      0.60      0.59      3589

